In [6]:
import numpy as np
from easymocap.mytools.camera_utils import write_camera, read_camera
import json
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

folder = 'data/wildtrack/'
output_k3d_folder = 'output/wildtrack/output-keypoints3d_raw/keypoints3d_raw/'

subfolder_labels = os.path.join(folder, 'output-keypoints3d/label')
pids = [0,1,2,3,7,9,10]
# join the masks together and invert them
cameras = read_camera("data/wildtrack/intri.yml", "data/wildtrack/extri.yml", [1,2,3,4,5,6,7])

def load_mask(path):
    return cv2.imread(path, cv2.IMREAD_GRAYSCALE)

def save_mask(mask, label_path, pid, cam, frame):
    os.makedirs(os.path.join(label_path, cam), exist_ok=True)
    path = os.path.join(label_path, cam, frame + "_" + str(pid) + ".png")
    cv2.imwrite(path, mask)
    
kintree = [[ 1,  0],
[ 2,  1],
[ 3,  2],
[ 4,  3],
[ 5,  1],
[ 6,  5],
[ 7,  6],
[ 8,  1],
[ 9,  8],
[10,  9],
[11, 10],
[12,  8],
[13, 12],
[14, 13],
[15,  0],
[16,  0],
[17, 15],
[18, 16],
[19, 14],
[20, 19],
[21, 14],
[22, 11],
[23, 22],
[24, 11]]


plt.figure(figsize=(10, 10))

for file in tqdm(os.listdir(output_k3d_folder)):
    file_path = os.path.join(output_k3d_folder, file)
    with open(file_path) as f:
        keypoints = json.load(f)
    all_kp3d = np.array(keypoints)
    
    for person in all_kp3d:
        for cam in cameras["basenames"]:
            P = cameras[cam]['P']
            label_img = np.zeros((1080, 1920, 1), dtype=np.uint8)
            pid = person['id']
            
            kp3s = person['keypoints3d']
            
            for j1,j2 in kintree:
                x1, y1, z1, prob1 = kp3s[j1]
                x2, y2, z2, prob2 = kp3s[j2]
                
                xy1 = np.dot(P, np.array([x1, y1, z1, 1]))
                xy2 = np.dot(P, np.array([x2, y2, z2, 1]))
                xy1 = xy1[:2] / xy1[2]
                xy2 = xy2[:2] / xy2[2]
                xy1 = xy1.astype(int)
                xy2 = xy2.astype(int)
                prob = (prob1 + prob2) / 2 if prob1 > 0 and prob2 > 0 else 0
                if prob > 0:
                    cv2.line(label_img, (xy1[0], xy1[1]), (xy2[0], xy2[1]), int(prob * 255), 10)
            save_mask(label_img, subfolder_labels, pid, cam, file.split('.')[0])

            
        
    

100%|██████████| 300/300 [01:08<00:00,  4.38it/s]


<Figure size 1000x1000 with 0 Axes>